**Module 1: Setup and Dependencies**

In [ ]:
# Module 1: Setup and Dependencies
# Copy and run this first

!pip install langchain langgraph openai pandas plotly streamlit-agraph networkx matplotlib seaborn -q

import os
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional
import uuid
from dataclasses import dataclass, asdict
from enum import Enum
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

# LangGraph imports
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict

# Set your OpenAI API key
if not os.getenv("OPENAI_API_KEY"):
   api_key = getpass.getpass("Enter your OpenAI API key: ")
   os.environ["OPENAI_API_KEY"] = api_key

print("✅ All dependencies installed and imported successfully!")
print("⚠️  Don't forget to set your OpenAI API key in the cell above")

**Module 2: Data Structures and Enums**

In [ ]:
# Module 2: Data Structures and Enums
# Core data models for the system

class MessageType(Enum):
    REQUEST_VALIDATION = "REQUEST_VALIDATION"
    REQUEST_APPROVAL = "REQUEST_APPROVAL"
    VALIDATE_BANK = "VALIDATE_BANK"
    RECONCILE_GAP = "RECONCILE_GAP"
    ASK_POLICY = "ASK_POLICY"
    APPROVE = "APPROVE"
    HOLD = "HOLD"
    SCHEDULE_PAYMENT = "SCHEDULE_PAYMENT"
    UPDATE_FORECAST = "UPDATE_FORECAST"

class Priority(Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    CRITICAL = "critical"

class Status(Enum):
    PENDING = "pending"
    APPROVED = "approved"
    REJECTED = "rejected"
    ON_HOLD = "on_hold"
    REQUIRES_HUMAN = "requires_human"

@dataclass
class Invoice:
    invoice_id: str
    vendor_id: str
    vendor_name: str
    amount: float
    currency: str
    invoice_date: str
    due_date: str
    po_number: str
    description: str
    line_items: List[Dict[str, Any]]
    bank_details: Dict[str, str]

@dataclass
class PurchaseOrder:
    po_number: str
    vendor_id: str
    total_amount: float
    currency: str
    line_items: List[Dict[str, Any]]
    approval_status: str
    created_date: str

@dataclass
class Vendor:
    vendor_id: str
    name: str
    risk_rating: str  # LOW, MEDIUM, HIGH
    bank_details: Dict[str, str]
    last_bank_change: Optional[str]
    payment_terms: int
    discount_terms: Optional[str]

@dataclass
class AgentMessage:
    from_agent: str
    to_agent: str
    message_type: MessageType
    content: Dict[str, Any]
    timestamp: str
    confidence_score: float
    citations: List[str]

@dataclass
class Decision:
    agent: str
    decision: str
    reasoning: str
    confidence: float
    evidence: List[str]
    timestamp: str

class WorkflowState(TypedDict):
    invoice: Dict[str, Any]
    po_data: Dict[str, Any]
    vendor_data: Dict[str, Any]
    messages: List[Dict[str, Any]]
    decisions: List[Dict[str, Any]]
    current_status: str
    requires_human: bool
    risk_score: float
    next_agent: str
    final_decision: Optional[str]
    audit_trail: List[str]

print("✅ Data structures and enums defined successfully!")

**Module 3: Mock Data Generation**

In [ ]:
# Module 3: Mock Data Generation
# Generate realistic test data for the system

import random
from datetime import datetime, timedelta

class MockDataGenerator:
    def __init__(self):
        self.vendors = self._create_vendors()
        self.policies = self._create_policies()

    def _create_vendors(self):
        return {
            "VEN001": Vendor(
                vendor_id="VEN001",
                name="Acme Office Supplies",
                risk_rating="LOW",
                bank_details={"account": "12345678", "routing": "987654321", "bank_name": "First Bank"},
                last_bank_change=None,
                payment_terms=30,
                discount_terms="2/10 net 30"
            ),
            "VEN002": Vendor(
                vendor_id="VEN002",
                name="Global Tech Solutions",
                risk_rating="MEDIUM",
                bank_details={"account": "87654321", "routing": "123456789", "bank_name": "Tech Credit Union"},
                last_bank_change="2024-01-15",
                payment_terms=45,
                discount_terms="1.5/15 net 45"
            ),
            "VEN003": Vendor(
                vendor_id="VEN003",
                name="International Consulting Corp",
                risk_rating="HIGH",
                bank_details={"account": "11223344", "routing": "556677889", "bank_name": "Offshore Bank"},
                last_bank_change="2024-02-01",
                payment_terms=60,
                discount_terms=None
            )
        }

    def _create_policies(self):
        return {
            "approval_limits": {
                "auto_approve": 5000,
                "manager_approve": 25000,
                "director_approve": 100000
            },
            "discount_capture": {
                "min_discount_percent": 1.0,
                "priority_suppliers": ["VEN001", "VEN002"]
            },
            "risk_thresholds": {
                "high_risk_review": 10000,
                "bank_change_verify": True,
                "sanctions_check": True
            }
        }

    def create_invoice(self, scenario="clean"):
        scenarios = {
            "clean": self._create_clean_invoice,
            "po_mismatch": self._create_po_mismatch_invoice,
            "high_value": self._create_high_value_invoice,
            "bank_change": self._create_bank_change_invoice,
            "risk_vendor": self._create_risk_vendor_invoice
        }
        return scenarios[scenario]()

    def _create_clean_invoice(self):
        vendor = self.vendors["VEN001"]
        invoice_id = f"INV-{random.randint(1000, 9999)}"
        po_number = f"PO-{random.randint(100, 999)}"

        line_items = [
            {"description": "Office Chair", "quantity": 5, "unit_price": 150.00, "total": 750.00},
            {"description": "Desk Supplies", "quantity": 10, "unit_price": 25.00, "total": 250.00}
        ]

        invoice = Invoice(
            invoice_id=invoice_id,
            vendor_id=vendor.vendor_id,
            vendor_name=vendor.name,
            amount=1000.00,
            currency="USD",
            invoice_date=(datetime.now() - timedelta(days=5)).strftime("%Y-%m-%d"),
            due_date=(datetime.now() + timedelta(days=25)).strftime("%Y-%m-%d"),
            po_number=po_number,
            description="Office furniture and supplies",
            line_items=line_items,
            bank_details=vendor.bank_details
        )

        po = PurchaseOrder(
            po_number=po_number,
            vendor_id=vendor.vendor_id,
            total_amount=1000.00,
            currency="USD",
            line_items=line_items,
            approval_status="APPROVED",
            created_date=(datetime.now() - timedelta(days=10)).strftime("%Y-%m-%d")
        )

        return invoice, po, vendor

    def _create_po_mismatch_invoice(self):
        vendor = self.vendors["VEN002"]
        invoice_id = f"INV-{random.randint(1000, 9999)}"
        po_number = f"PO-{random.randint(100, 999)}"

        invoice_line_items = [
            {"description": "Software License", "quantity": 10, "unit_price": 500.00, "total": 5000.00}
        ]

        po_line_items = [
            {"description": "Software License", "quantity": 8, "unit_price": 500.00, "total": 4000.00}
        ]

        invoice = Invoice(
            invoice_id=invoice_id,
            vendor_id=vendor.vendor_id,
            vendor_name=vendor.name,
            amount=5000.00,
            currency="USD",
            invoice_date=(datetime.now() - timedelta(days=3)).strftime("%Y-%m-%d"),
            due_date=(datetime.now() + timedelta(days=42)).strftime("%Y-%m-%d"),
            po_number=po_number,
            description="Software licensing",
            line_items=invoice_line_items,
            bank_details=vendor.bank_details
        )

        po = PurchaseOrder(
            po_number=po_number,
            vendor_id=vendor.vendor_id,
            total_amount=4000.00,
            currency="USD",
            line_items=po_line_items,
            approval_status="APPROVED",
            created_date=(datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d")
        )

        return invoice, po, vendor

    def _create_high_value_invoice(self):
        vendor = self.vendors["VEN002"]
        invoice_id = f"INV-{random.randint(1000, 9999)}"
        po_number = f"PO-{random.randint(100, 999)}"

        line_items = [
            {"description": "Enterprise Software Implementation", "quantity": 1, "unit_price": 75000.00, "total": 75000.00}
        ]

        invoice = Invoice(
            invoice_id=invoice_id,
            vendor_id=vendor.vendor_id,
            vendor_name=vendor.name,
            amount=75000.00,
            currency="USD",
            invoice_date=(datetime.now() - timedelta(days=2)).strftime("%Y-%m-%d"),
            due_date=(datetime.now() + timedelta(days=43)).strftime("%Y-%m-%d"),
            po_number=po_number,
            description="Major software implementation project",
            line_items=line_items,
            bank_details=vendor.bank_details
        )

        po = PurchaseOrder(
            po_number=po_number,
            vendor_id=vendor.vendor_id,
            total_amount=75000.00,
            currency="USD",
            line_items=line_items,
            approval_status="APPROVED",
            created_date=(datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")
        )

        return invoice, po, vendor

    def _create_bank_change_invoice(self):
        vendor = self.vendors["VEN003"]
        invoice_id = f"INV-{random.randint(1000, 9999)}"
        po_number = f"PO-{random.randint(100, 999)}"

        line_items = [
            {"description": "Consulting Services", "quantity": 1, "unit_price": 15000.00, "total": 15000.00}
        ]

        invoice = Invoice(
            invoice_id=invoice_id,
            vendor_id=vendor.vendor_id,
            vendor_name=vendor.name,
            amount=15000.00,
            currency="USD",
            invoice_date=(datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d"),
            due_date=(datetime.now() + timedelta(days=59)).strftime("%Y-%m-%d"),
            po_number=po_number,
            description="Management consulting services",
            line_items=line_items,
            bank_details=vendor.bank_details
        )

        po = PurchaseOrder(
            po_number=po_number,
            vendor_id=vendor.vendor_id,
            total_amount=15000.00,
            currency="USD",
            line_items=line_items,
            approval_status="APPROVED",
            created_date=(datetime.now() - timedelta(days=20)).strftime("%Y-%m-%d")
        )

        return invoice, po, vendor

    def _create_risk_vendor_invoice(self):
        return self._create_bank_change_invoice()  # Same as bank change for simplicity

# Initialize the data generator
data_generator = MockDataGenerator()

# Test data generation
test_scenarios = ["clean", "po_mismatch", "high_value", "bank_change", "risk_vendor"]
print("✅ Mock data generator created successfully!")
print(f"📊 Available test scenarios: {test_scenarios}")
print(f"🏪 Created {len(data_generator.vendors)} vendors")
print(f"📋 Policy framework loaded")

**Module 4: Base Agent Class**

In [ ]:
# Module 4: Base Agent Class
# Foundation class for all AI agents

class BaseAgent:
    def __init__(self, name: str, role: str, model_name: str = "gpt-3.5-turbo"):
        self.name = name
        self.role = role
        self.llm = ChatOpenAI(model=model_name, temperature=0.1)
        self.decisions = []
        self.message_history = []

    def create_system_prompt(self) -> str:
        """Override this method in each specialized agent"""
        return f"""You are {self.name}, a specialized AI agent for {self.role}.
        You make decisions based on evidence and provide clear reasoning.
        Always include confidence scores (0.0-1.0) and cite your sources.
        Respond in structured JSON format."""

    def log_decision(self, decision: Decision):
        """Log agent decisions for audit trail"""
        self.decisions.append(decision)

    def log_message(self, message: AgentMessage):
        """Log inter-agent messages"""
        self.message_history.append(message)

    def make_decision(self, context: Dict[str, Any], query: str) -> Decision:
        """Base decision-making method"""
        system_prompt = self.create_system_prompt()

        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=f"""
            Context: {json.dumps(context, indent=2)}

            Query: {query}

            Please provide your decision in this JSON format:
            {{
                "decision": "your decision (APPROVE/REJECT/HOLD/NEEDS_REVIEW)",
                "reasoning": "detailed explanation",
                "confidence": 0.95,
                "evidence": ["source1", "source2"],
                "recommendations": ["action1", "action2"],
                "risk_factors": ["risk1", "risk2"]
            }}
            """)
        ]

        try:
            response = self.llm(messages)
            # Parse JSON response
            decision_data = json.loads(response.content)

            decision = Decision(
                agent=self.name,
                decision=decision_data.get("decision", "NEEDS_REVIEW"),
                reasoning=decision_data.get("reasoning", "No reasoning provided"),
                confidence=decision_data.get("confidence", 0.5),
                evidence=decision_data.get("evidence", []),
                timestamp=datetime.now().isoformat()
            )

            self.log_decision(decision)
            return decision

        except Exception as e:
            # Fallback decision if JSON parsing fails
            decision = Decision(
                agent=self.name,
                decision="NEEDS_REVIEW",
                reasoning=f"Error processing decision: {str(e)}",
                confidence=0.0,
                evidence=["error_occurred"],
                timestamp=datetime.now().isoformat()
            )
            self.log_decision(decision)
            return decision

    def send_message(self, to_agent: str, message_type: MessageType,
                    content: Dict[str, Any], confidence: float = 1.0) -> AgentMessage:
        """Send structured message to another agent"""
        message = AgentMessage(
            from_agent=self.name,
            to_agent=to_agent,
            message_type=message_type,
            content=content,
            timestamp=datetime.now().isoformat(),
            confidence_score=confidence,
            citations=content.get("citations", [])
        )

        self.log_message(message)
        return message

    def get_audit_trail(self) -> List[Dict[str, Any]]:
        """Get complete audit trail for this agent"""
        trail = []

        for decision in self.decisions:
            trail.append({
                "type": "decision",
                "timestamp": decision.timestamp,
                "agent": decision.agent,
                "content": asdict(decision)
            })

        for message in self.message_history:
            trail.append({
                "type": "message",
                "timestamp": message.timestamp,
                "from": message.from_agent,
                "to": message.to_agent,
                "content": asdict(message)
            })

        return sorted(trail, key=lambda x: x["timestamp"])

print("✅ Base Agent class created successfully!")
print("🤖 Ready to create specialized agents")

**Module 5: Specialized Agents**

In [ ]:
# Module 5: Specialized Agents
# AP Agent, Compliance Agent, Payment Agent, Forecasting Agent

class APAgent(BaseAgent):
    def __init__(self):
        super().__init__("AP_Agent", "Accounts Payable Processing", "gpt-3.5-turbo")
        self.tolerance_percent = 0.05  # 5% tolerance for amount mismatches

    def create_system_prompt(self) -> str:
        return """You are the AP Agent, responsible for invoice validation and processing.

        Your responsibilities:
        - Validate invoice data completeness and accuracy
        - Match invoices against Purchase Orders (PO) and Goods Received Notes (GRN)
        - Identify discrepancies in amounts, quantities, descriptions
        - Flag missing approvals or incomplete documentation
        - Apply tolerance rules for minor variances (5% default)

        Decision criteria:
        - APPROVE: Perfect match or within tolerance
        - HOLD: Discrepancies outside tolerance, missing PO/GRN
        - NEEDS_REVIEW: Complex cases requiring human judgment

        Always provide specific line-item analysis and cite exact discrepancies."""

    def validate_invoice(self, invoice: Invoice, po: PurchaseOrder, vendor: Vendor) -> Decision:
        """Main invoice validation logic"""
        context = {
            "invoice": asdict(invoice),
            "purchase_order": asdict(po),
            "vendor": asdict(vendor),
            "tolerance_percent": self.tolerance_percent
        }

        query = f"""Validate invoice {invoice.invoice_id} against PO {po.po_number}.

        Check for:
        1. Amount matching (within {self.tolerance_percent*100}% tolerance)
        2. Line item quantities and descriptions
        3. Vendor ID consistency
        4. PO approval status
        5. Invoice date reasonableness

        Provide detailed analysis of any discrepancies found."""

        return self.make_decision(context, query)

    def calculate_discrepancy(self, invoice_amount: float, po_amount: float) -> Dict[str, Any]:
        """Calculate and categorize amount discrepancies"""
        difference = abs(invoice_amount - po_amount)
        percent_diff = (difference / po_amount) * 100 if po_amount > 0 else 100

        return {
            "difference_amount": difference,
            "percent_difference": percent_diff,
            "within_tolerance": percent_diff <= (self.tolerance_percent * 100),
            "invoice_amount": invoice_amount,
            "po_amount": po_amount
        }

class ComplianceAgent(BaseAgent):
    def __init__(self):
        super().__init__("Compliance_Agent", "Risk and Compliance Management", "gpt-3.5-turbo")
        self.sanctions_list = ["BadVendor Inc", "Risky Corp", "Sanctioned Ltd"]  # Mock sanctions
        self.high_risk_countries = ["Country A", "Country B"]  # Mock high-risk countries

    def create_system_prompt(self) -> str:
        return """You are the Compliance Agent, responsible for risk assessment and policy enforcement.

        Your responsibilities:
        - Screen vendors against sanctions and PEP lists
        - Validate payment amounts against approval limits
        - Check compliance with company policies and procedures
        - Assess vendor risk ratings and payment history
        - Flag suspicious patterns or unusual transactions
        - Verify bank detail changes and validate legitimacy

        Decision criteria:
        - APPROVE: All compliance checks pass, low risk
        - HOLD: Policy violations, high risk indicators, requires verification
        - NEEDS_REVIEW: Complex compliance issues requiring human judgment

        Always provide risk scores (0-10 scale) and specific compliance citations."""

    def assess_compliance(self, invoice: Invoice, vendor: Vendor, policies: Dict[str, Any]) -> Decision:
        """Main compliance assessment logic"""
        context = {
            "invoice": asdict(invoice),
            "vendor": asdict(vendor),
            "policies": policies,
            "sanctions_list": self.sanctions_list,
            "high_risk_countries": self.high_risk_countries
        }

        query = f"""Assess compliance for invoice {invoice.invoice_id} from vendor {vendor.name}.

        Check for:
        1. Sanctions screening (vendor not in sanctions list)
        2. Approval limits (amount ${invoice.amount} vs policy limits)
        3. Vendor risk rating ({vendor.risk_rating})
        4. Bank details change verification needed
        5. Payment terms compliance
        6. Unusual patterns or red flags

        Provide risk score (0-10) and specific compliance findings."""

        return self.make_decision(context, query)

    def check_sanctions(self, vendor_name: str) -> Dict[str, Any]:
        """Check if vendor is on sanctions list"""
        is_sanctioned = vendor_name in self.sanctions_list
        return {
            "is_sanctioned": is_sanctioned,
            "vendor_name": vendor_name,
            "sanctions_list_checked": True,
            "risk_level": "HIGH" if is_sanctioned else "LOW"
        }

    def validate_bank_change(self, vendor: Vendor) -> Dict[str, Any]:
        """Validate recent bank detail changes"""
        needs_verification = False
        risk_score = 1  # Low risk by default

        if vendor.last_bank_change:
            change_date = datetime.strptime(vendor.last_bank_change, "%Y-%m-%d")
            days_since_change = (datetime.now() - change_date).days

            # Flag if bank change was recent (within 30 days)
            if days_since_change <= 30:
                needs_verification = True
                risk_score = 7 if vendor.risk_rating == "HIGH" else 5

        return {
            "needs_verification": needs_verification,
            "days_since_change": days_since_change if vendor.last_bank_change else None,
            "risk_score": risk_score,
            "verification_required": needs_verification and risk_score >= 5
        }

class PaymentAgent(BaseAgent):
    def __init__(self):
        super().__init__("Payment_Agent", "Payment Processing and Optimization", "gpt-3.5-turbo")
        self.current_cash_balance = 500000  # Mock cash balance

    def create_system_prompt(self) -> str:
        return """You are the Payment Agent, responsible for payment scheduling and optimization.

        Your responsibilities:
        - Schedule payments based on due dates and cash flow
        - Optimize for early payment discounts when beneficial
        - Validate bank details and prevent duplicate payments
        - Balance cash constraints with payment obligations
        - Ensure payment method compliance and security

        Decision criteria:
        - APPROVE: Payment scheduled optimally, cash available, details verified
        - HOLD: Cash constraints, bank details issues, duplicate risk
        - NEEDS_REVIEW: Complex optimization scenarios, high-value payments

        Always consider cash flow impact and discount opportunities."""

    def schedule_payment(self, invoice: Invoice, vendor: Vendor, cash_constraints: Dict[str, Any]) -> Decision:
        """Main payment scheduling logic"""
        context = {
            "invoice": asdict(invoice),
            "vendor": asdict(vendor),
            "current_cash_balance": self.current_cash_balance,
            "cash_constraints": cash_constraints
        }

        query = f"""Schedule payment for invoice {invoice.invoice_id} amount ${invoice.amount}.

        Consider:
        1. Due date: {invoice.due_date}
        2. Discount terms: {vendor.discount_terms}
        3. Current cash balance: ${self.current_cash_balance}
        4. Vendor payment terms: {vendor.payment_terms} days
        5. Bank details verification status

        Optimize for discount capture while managing cash flow.
        Recommend payment date and method."""

        return self.make_decision(context, query)

    def calculate_discount_benefit(self, invoice: Invoice, vendor: Vendor) -> Dict[str, Any]:
        """Calculate early payment discount benefits"""
        if not vendor.discount_terms:
            return {"has_discount": False, "discount_amount": 0, "net_benefit": 0}

        # Parse discount terms like "2/10 net 30"
        try:
            parts = vendor.discount_terms.split()
            discount_info = parts[0].split('/')
            discount_percent = float(discount_info[0])
            discount_days = int(discount_info[1])

            discount_amount = invoice.amount * (discount_percent / 100)
            days_to_discount = discount_days

            # Simple ROI calculation (annualized)
            days_early = vendor.payment_terms - discount_days
            if days_early > 0:
                annual_rate = (discount_percent / 100) * (365 / days_early)
                net_benefit = discount_amount  # Simplified
            else:
                annual_rate = 0
                net_benefit = 0

            return {
                "has_discount": True,
                "discount_percent": discount_percent,
                "discount_amount": discount_amount,
                "discount_deadline_days": discount_days,
                "net_benefit": net_benefit,
                "annual_rate": annual_rate,
                "recommended": net_benefit > 0 and self.current_cash_balance > invoice.amount
            }
        except:
            return {"has_discount": False, "discount_amount": 0, "net_benefit": 0, "error": "Could not parse discount terms"}

class ForecastingAgent(BaseAgent):
    def __init__(self):
        super().__init__("Forecasting_Agent", "Cash Flow Forecasting", "gpt-3.5-turbo")
        self.forecast_horizon_days = 90

    def create_system_prompt(self) -> str:
        return """You are the Forecasting Agent, responsible for cash flow analysis and predictions.

        Your responsibilities:
        - Update cash flow forecasts based on approved payments
        - Analyze liquidity impacts of payment decisions
        - Identify cash constraints and optimization opportunities
        - Provide early warning of cash flow issues
        - Recommend payment timing adjustments

        Decision criteria:
        - APPROVE: Sufficient cash flow, no liquidity concerns
        - HOLD: Cash flow constraints, liquidity risk
        - NEEDS_REVIEW: Complex cash flow scenarios, requires CFO input

        Always provide cash flow projections and liquidity analysis."""

    def update_forecast(self, invoice: Invoice, payment_date: str, current_commitments: List[Dict]) -> Decision:
        """Update cash flow forecast with new payment"""
        context = {
            "invoice": asdict(invoice),
            "payment_date": payment_date,
            "current_commitments": current_commitments,
            "forecast_horizon": self.forecast_horizon_days
        }

        query = f"""Update cash flow forecast for payment of ${invoice.amount} on {payment_date}.

        Analyze:
        1. Impact on daily cash positions over next {self.forecast_horizon_days} days
        2. Liquidity risk assessment
        3. Cash constraint identification
        4. Alternative timing recommendations
        5. Early warning indicators

        Provide updated forecast summary and recommendations."""

        return self.make_decision(context, query)

    def assess_liquidity_impact(self, payment_amount: float, payment_date: str) -> Dict[str, Any]:
        """Assess liquidity impact of proposed payment"""
        # Mock liquidity calculation
        projected_balance = self.current_cash_balance - payment_amount

        # Mock daily cash flows (would be from real data in production)
        daily_inflows = 50000  # Average daily receipts
        daily_outflows = 45000  # Average daily payments

        days_until_payment = (datetime.strptime(payment_date, "%Y-%m-%d") - datetime.now()).days
        projected_balance_at_payment = (
            self.current_cash_balance +
            (daily_inflows - daily_outflows) * days_until_payment -
            payment_amount
        )

        return {
            "current_balance": self.current_cash_balance,
            "payment_amount": payment_amount,
            "projected_balance": projected_balance_at_payment,
            "liquidity_risk": "HIGH" if projected_balance_at_payment < 100000 else "LOW",
            "days_until_payment": days_until_payment,
            "recommendation": "APPROVE" if projected_balance_at_payment > 200000 else "DEFER"
        }

# Create instances of all agents
ap_agent = APAgent()
compliance_agent = ComplianceAgent()
payment_agent = PaymentAgent()
forecasting_agent = ForecastingAgent()

print("✅ All specialized agents created successfully!")
print("🤖 AP Agent - Ready for invoice validation")
print("🛡️  Compliance Agent - Ready for risk assessment")
print("💳 Payment Agent - Ready for payment scheduling")
print("📈 Forecasting Agent - Ready for cash flow analysis")

**Module 6: Supervisor Agent**

In [ ]:
# Module 6: Supervisor Agent
# Orchestrates workflow and makes final decisions

class SupervisorAgent(BaseAgent):
    def __init__(self):
        super().__init__("Supervisor_Agent", "Workflow Orchestration and Decision Authority", "gpt-4")
        self.agents = {
            "ap": ap_agent,
            "compliance": compliance_agent,
            "payment": payment_agent,
            "forecasting": forecasting_agent
        }
        self.workflow_state = None
        self.escalation_thresholds = {
            "amount": 25000,  # Amounts above this need human approval
            "risk_score": 7,   # Risk scores above this need human review
            "confidence": 0.7  # Decisions below this confidence need review
        }

    def create_system_prompt(self) -> str:
        return """You are the Supervisor Agent, the orchestrator and final decision authority.

        Your responsibilities:
        - Coordinate workflow between specialized agents
        - Resolve conflicts when agents disagree
        - Make final approve/reject decisions
        - Escalate to humans when thresholds are exceeded
        - Ensure audit trail completeness and compliance
        - Optimize overall process efficiency

        Decision authority:
        - You can override agent recommendations with clear justification
        - You must escalate high-value or high-risk transactions
        - You ensure all required validations are completed
        - You maintain final accountability for all decisions

        Always provide comprehensive reasoning for final decisions."""

    def orchestrate_workflow(self, invoice: Invoice, po: PurchaseOrder, vendor: Vendor) -> Dict[str, Any]:
        """Main workflow orchestration method"""
        print(f"\n🎯 Supervisor orchestrating workflow for Invoice {invoice.invoice_id}")

        # Initialize workflow state
        workflow_result = {
            "invoice_id": invoice.invoice_id,
            "vendor_name": vendor.name,
            "amount": invoice.amount,
            "agent_decisions": {},
            "final_decision": None,
            "requires_human": False,
            "reasoning": "",
            "audit_trail": [],
            "processing_time": datetime.now().isoformat()
        }

        try:
            # Step 1: AP Agent validation
            print("  📋 Step 1: Invoice validation by AP Agent...")
            ap_decision = ap_agent.validate_invoice(invoice, po, vendor)
            workflow_result["agent_decisions"]["ap"] = asdict(ap_decision)
            workflow_result["audit_trail"].append(f"AP Agent: {ap_decision.decision} - {ap_decision.reasoning}")

            if ap_decision.decision == "REJECT":
                return self._finalize_workflow(workflow_result, "REJECTED", "Failed AP validation", ap_decision)

            # Step 2: Compliance assessment
            print("  🛡️  Step 2: Compliance assessment...")
            compliance_decision = compliance_agent.assess_compliance(invoice, vendor, data_generator.policies)
            workflow_result["agent_decisions"]["compliance"] = asdict(compliance_decision)
            workflow_result["audit_trail"].append(f"Compliance Agent: {compliance_decision.decision} - {compliance_decision.reasoning}")

            if compliance_decision.decision == "HOLD":
                return self._finalize_workflow(workflow_result, "ON_HOLD", "Compliance hold", compliance_decision)

            # Step 3: Payment scheduling (if compliance passes)
            if compliance_decision.decision in ["APPROVE", "NEEDS_REVIEW"]:
                print("  💳 Step 3: Payment scheduling...")
                payment_decision = payment_agent.schedule_payment(invoice, vendor, {"cash_available": True})
                workflow_result["agent_decisions"]["payment"] = asdict(payment_decision)
                workflow_result["audit_trail"].append(f"Payment Agent: {payment_decision.decision} - {payment_decision.reasoning}")

                # Step 4: Forecasting update
                print("  📈 Step 4: Cash flow forecast update...")
                forecast_decision = forecasting_agent.update_forecast(invoice,
                    (datetime.now() + timedelta(days=vendor.payment_terms)).strftime("%Y-%m-%d"), [])
                workflow_result["agent_decisions"]["forecasting"] = asdict(forecast_decision)
                workflow_result["audit_trail"].append(f"Forecasting Agent: {forecast_decision.decision} - {forecast_decision.reasoning}")

            # Step 5: Supervisor final decision
            print("  🎯 Step 5: Supervisor final decision...")
            final_decision = self._make_final_decision(workflow_result, invoice, vendor)

            return final_decision

        except Exception as e:
            workflow_result["final_decision"] = "ERROR"
            workflow_result["reasoning"] = f"Workflow error: {str(e)}"
            workflow_result["requires_human"] = True
            return workflow_result

    def _make_final_decision(self, workflow_result: Dict[str, Any], invoice: Invoice, vendor: Vendor) -> Dict[str, Any]:
        """Make the final supervisory decision"""

        # Collect all agent decisions
        agent_decisions = workflow_result["agent_decisions"]

        # Check escalation criteria
        needs_escalation = self._check_escalation_criteria(invoice, agent_decisions)

        if needs_escalation:
            return self._finalize_workflow(workflow_result, "REQUIRES_HUMAN",
                                         "Escalated due to amount, risk, or confidence thresholds", None)

        # Count approve vs hold/reject decisions
        approvals = sum(1 for decision in agent_decisions.values()
                       if decision.get("decision") == "APPROVE")
        holds = sum(1 for decision in agent_decisions.values()
                   if decision.get("decision") in ["HOLD", "REJECT"])

        # Supervisor decision logic
        context = {
            "workflow_state": workflow_result,
            "agent_decisions": agent_decisions,
            "escalation_check": needs_escalation,
            "approval_count": approvals,
            "hold_count": holds
        }

        query = f"""Make final decision for invoice {invoice.invoice_id}.

        Agent decisions summary:
        - Approvals: {approvals}
        - Holds/Rejects: {holds}
        - Requires escalation: {needs_escalation}

        Consider:
        1. Overall risk assessment
        2. Agent confidence levels
        3. Business impact and urgency
        4. Compliance requirements
        5. Cash flow implications

        Provide final APPROVE/REJECT/HOLD decision with comprehensive reasoning."""

        supervisor_decision = self.make_decision(context, query)

        return self._finalize_workflow(workflow_result, supervisor_decision.decision,
                                     supervisor_decision.reasoning, supervisor_decision)

    def _check_escalation_criteria(self, invoice: Invoice, agent_decisions: Dict[str, Any]) -> bool:
        """Check if transaction meets escalation criteria"""

        # High value check
        if invoice.amount > self.escalation_thresholds["amount"]:
            return True

        # Low confidence check
        for decision in agent_decisions.values():
            if decision.get("confidence", 1.0) < self.escalation_thresholds["confidence"]:
                return True

        # High risk check - would need risk score from compliance agent
        # For now, simplified based on vendor risk rating
        # if vendor.risk_rating == "HIGH" and invoice.amount > 10000:
        #     return True

        return False

    def _finalize_workflow(self, workflow_result: Dict[str, Any], decision: str,
                          reasoning: str, supervisor_decision: Decision = None) -> Dict[str, Any]:
        """Finalize workflow with decision and audit trail"""

        workflow_result["final_decision"] = decision
        workflow_result["reasoning"] = reasoning
        workflow_result["requires_human"] = decision == "REQUIRES_HUMAN"
        workflow_result["completed_at"] = datetime.now().isoformat()

        if supervisor_decision:
            workflow_result["supervisor_decision"] = asdict(supervisor_decision)
            workflow_result["audit_trail"].append(f"Supervisor: {decision} - {reasoning}")

        # Add summary metrics
        workflow_result["summary"] = {
            "total_agents_consulted": len(workflow_result["agent_decisions"]),
            "unanimous_decision": self._is_unanimous_decision(workflow_result["agent_decisions"]),
            "processing_duration": "< 1 minute",  # Mock timing
            "escalation_triggered": workflow_result["requires_human"]
        }

        return workflow_result

    def _is_unanimous_decision(self, agent_decisions: Dict[str, Any]) -> bool:
        """Check if all agents made the same decision"""
        if not agent_decisions:
            return True

        decisions = [decision.get("decision") for decision in agent_decisions.values()]
        return len(set(decisions)) == 1

    def get_workflow_summary(self, workflow_result: Dict[str, Any]) -> str:
        """Generate human-readable workflow summary"""
        decision = workflow_result["final_decision"]
        amount = workflow_result["amount"]
        vendor = workflow_result["vendor_name"]

        summary = f"""
        🏁 WORKFLOW COMPLETE - Invoice {workflow_result['invoice_id']}

        💰 Amount: ${amount:,.2f}
        🏪 Vendor: {vendor}
        ✅ Decision: {decision}
        🧠 Reasoning: {workflow_result['reasoning']}

        📊 Agent Decisions:
        """

        for agent_name, decision_data in workflow_result["agent_decisions"].items():
            summary += f"   • {agent_name.upper()}: {decision_data.get('decision')} (confidence: {decision_data.get('confidence', 0):.2f})\n"

        if workflow_result["requires_human"]:
            summary += "\n⚠️  HUMAN APPROVAL REQUIRED"

        return summary

# Create supervisor agent
supervisor_agent = SupervisorAgent()

print("✅ Supervisor Agent created successfully!")
print("🎯 Ready to orchestrate multi-agent workflows")
print("🚦 Escalation thresholds configured:")
print(f"   • Amount threshold: ${supervisor_agent.escalation_thresholds['amount']:,}")
print(f"   • Risk score threshold: {supervisor_agent.escalation_thresholds['risk_score']}")
print(f"   • Confidence threshold: {supervisor_agent.escalation_thresholds['confidence']}")

**Module 7: LangGraph Workflow Integration**

In [ ]:
# Module 7: LangGraph Workflow Integration
# Implements the workflow using LangGraph state management

from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict
import json

# Define the workflow state schema
class FinanceWorkflowState(TypedDict):
    # Core invoice data
    invoice: Dict[str, Any]
    po_data: Dict[str, Any]
    vendor_data: Dict[str, Any]

    # Workflow tracking
    current_step: str
    next_agent: str
    workflow_complete: bool
    requires_human_approval: bool

    # Agent decisions and messages
    agent_decisions: Dict[str, Any]
    agent_messages: List[Dict[str, Any]]

    # Final outcomes
    final_decision: Optional[str]
    approval_status: str
    risk_assessment: Dict[str, Any]

    # Audit and compliance
    audit_trail: List[str]
    processing_metadata: Dict[str, Any]

    # Error handling
    errors: List[str]
    retry_count: int

class FinanceWorkflowGraph:
    def __init__(self):
        self.graph = StateGraph(FinanceWorkflowState)
        self.memory = MemorySaver()
        self._build_graph()
        self._compile_graph()

        # Workflow statistics
        self.workflow_stats = {
            "total_processed": 0,
            "approved": 0,
            "rejected": 0,
            "on_hold": 0,
            "requires_human": 0,
            "processing_times": [],
            "agent_performance": {}
        }

    def _build_graph(self):
        """Build the LangGraph workflow structure"""

        # Add nodes for each agent and workflow step
        self.graph.add_node("start", self._start_workflow)
        self.graph.add_node("ap_validation", self._ap_validation_step)
        self.graph.add_node("compliance_check", self._compliance_check_step)
        self.graph.add_node("payment_scheduling", self._payment_scheduling_step)
        self.graph.add_node("forecast_update", self._forecast_update_step)
        self.graph.add_node("supervisor_decision", self._supervisor_decision_step)
        self.graph.add_node("human_approval", self._human_approval_step)
        self.graph.add_node("finalize", self._finalize_workflow)

        # Define the workflow edges and routing logic
        self.graph.set_entry_point("start")

        # Start -> AP Validation
        self.graph.add_edge("start", "ap_validation")

        # AP Validation -> Compliance or End (if rejected)
        self.graph.add_conditional_edges(
            "ap_validation",
            self._route_after_ap,
            {
                "compliance": "compliance_check",
                "rejected": "finalize"
            }
        )

        # Compliance -> Payment or Hold
        self.graph.add_conditional_edges(
            "compliance_check",
            self._route_after_compliance,
            {
                "payment": "payment_scheduling",
                "hold": "finalize",
                "human_review": "human_approval"
            }
        )

        # Payment -> Forecasting
        self.graph.add_edge("payment_scheduling", "forecast_update")

        # Forecasting -> Supervisor Decision
        self.graph.add_edge("forecast_update", "supervisor_decision")

        # Supervisor -> Human Approval or Finalize
        self.graph.add_conditional_edges(
            "supervisor_decision",
            self._route_supervisor_decision,
            {
                "approve": "finalize",
                "human_review": "human_approval",
                "hold": "finalize"
            }
        )

        # Human Approval -> Finalize
        self.graph.add_edge("human_approval", "finalize")

        # Finalize -> END
        self.graph.add_edge("finalize", END)

    def _compile_graph(self):
        """Compile the graph for execution"""
        self.app = self.graph.compile(checkpointer=self.memory)
        print("✅ LangGraph workflow compiled successfully!")

    # Node functions
    def _start_workflow(self, state: FinanceWorkflowState) -> FinanceWorkflowState:
        """Initialize the workflow"""
        print(f"\n🚀 Starting workflow for Invoice {state['invoice']['invoice_id']}")

        state["current_step"] = "start"
        state["next_agent"] = "ap"
        state["workflow_complete"] = False
        state["requires_human_approval"] = False
        state["agent_decisions"] = {}
        state["agent_messages"] = []
        state["audit_trail"] = []
        state["errors"] = []
        state["retry_count"] = 0
        state["approval_status"] = "PROCESSING"
        state["processing_metadata"] = {
            "start_time": datetime.now().isoformat(),
            "workflow_version": "1.0"
        }

        state["audit_trail"].append("Workflow initiated")
        return state

    def _ap_validation_step(self, state: FinanceWorkflowState) -> FinanceWorkflowState:
        """AP Agent validation step"""
        print("  📋 AP Agent: Validating invoice...")

        try:
            # Convert dict back to objects for agent processing
            invoice = Invoice(**state["invoice"])
            po = PurchaseOrder(**state["po_data"])
            vendor = Vendor(**state["vendor_data"])

            # Run AP validation
            ap_decision = ap_agent.validate_invoice(invoice, po, vendor)

            # Store decision in state
            state["agent_decisions"]["ap"] = asdict(ap_decision)
            state["current_step"] = "ap_validation"
            state["audit_trail"].append(f"AP Agent: {ap_decision.decision} - {ap_decision.reasoning}")

            # Update risk assessment
            if "risk_assessment" not in state:
                state["risk_assessment"] = {}
            state["risk_assessment"]["ap_risk"] = "LOW" if ap_decision.decision == "APPROVE" else "MEDIUM"

            print(f"     ✓ AP Decision: {ap_decision.decision} (confidence: {ap_decision.confidence:.2f})")

        except Exception as e:
            state["errors"].append(f"AP validation error: {str(e)}")
            state["agent_decisions"]["ap"] = {
                "decision": "ERROR",
                "reasoning": f"Processing error: {str(e)}",
                "confidence": 0.0
            }

        return state

    def _compliance_check_step(self, state: FinanceWorkflowState) -> FinanceWorkflowState:
        """Compliance Agent assessment step"""
        print("  🛡️  Compliance Agent: Assessing risk and policy compliance...")

        try:
            invoice = Invoice(**state["invoice"])
            vendor = Vendor(**state["vendor_data"])

            # Run compliance assessment
            compliance_decision = compliance_agent.assess_compliance(
                invoice, vendor, data_generator.policies
            )

            # Store decision in state
            state["agent_decisions"]["compliance"] = asdict(compliance_decision)
            state["current_step"] = "compliance_check"
            state["audit_trail"].append(f"Compliance Agent: {compliance_decision.decision} - {compliance_decision.reasoning}")

            # Update risk assessment
            state["risk_assessment"]["compliance_risk"] = vendor.risk_rating
            state["risk_assessment"]["sanctions_clear"] = vendor.name not in compliance_agent.sanctions_list

            # Check bank validation if needed
            bank_validation = compliance_agent.validate_bank_change(vendor)
            state["risk_assessment"]["bank_validation"] = bank_validation

            print(f"     ✓ Compliance Decision: {compliance_decision.decision} (confidence: {compliance_decision.confidence:.2f})")

        except Exception as e:
            state["errors"].append(f"Compliance check error: {str(e)}")
            state["agent_decisions"]["compliance"] = {
                "decision": "ERROR",
                "reasoning": f"Processing error: {str(e)}",
                "confidence": 0.0
            }

        return state

    def _payment_scheduling_step(self, state: FinanceWorkflowState) -> FinanceWorkflowState:
        """Payment Agent scheduling step"""
        print("  💳 Payment Agent: Scheduling payment...")

        try:
            invoice = Invoice(**state["invoice"])
            vendor = Vendor(**state["vendor_data"])

            # Run payment scheduling
            payment_decision = payment_agent.schedule_payment(
                invoice, vendor, {"cash_available": True}
            )

            # Calculate discount benefits
            discount_analysis = payment_agent.calculate_discount_benefit(invoice, vendor)

            # Store decisions in state
            state["agent_decisions"]["payment"] = asdict(payment_decision)
            state["risk_assessment"]["payment_analysis"] = discount_analysis
            state["current_step"] = "payment_scheduling"
            state["audit_trail"].append(f"Payment Agent: {payment_decision.decision} - {payment_decision.reasoning}")

            print(f"     ✓ Payment Decision: {payment_decision.decision} (confidence: {payment_decision.confidence:.2f})")
            if discount_analysis.get("has_discount"):
                print(f"     💰 Discount available: {discount_analysis['discount_percent']}% = ${discount_analysis['discount_amount']:.2f}")

        except Exception as e:
            state["errors"].append(f"Payment scheduling error: {str(e)}")
            state["agent_decisions"]["payment"] = {
                "decision": "ERROR",
                "reasoning": f"Processing error: {str(e)}",
                "confidence": 0.0
            }

        return state

    def _forecast_update_step(self, state: FinanceWorkflowState) -> FinanceWorkflowState:
        """Forecasting Agent update step"""
        print("  📈 Forecasting Agent: Updating cash flow projections...")

        try:
            invoice = Invoice(**state["invoice"])
            vendor = Vendor(**state["vendor_data"])

            # Calculate payment date
            payment_date = (datetime.now() + timedelta(days=vendor.payment_terms)).strftime("%Y-%m-%d")

            # Run forecast update
            forecast_decision = forecasting_agent.update_forecast(invoice, payment_date, [])

            # Assess liquidity impact
            liquidity_analysis = forecasting_agent.assess_liquidity_impact(invoice.amount, payment_date)

            # Store decisions in state
            state["agent_decisions"]["forecasting"] = asdict(forecast_decision)
            state["risk_assessment"]["liquidity_analysis"] = liquidity_analysis
            state["current_step"] = "forecast_update"
            state["audit_trail"].append(f"Forecasting Agent: {forecast_decision.decision} - {forecast_decision.reasoning}")

            print(f"     ✓ Forecast Decision: {forecast_decision.decision} (confidence: {forecast_decision.confidence:.2f})")
            print(f"     📊 Liquidity Impact: {liquidity_analysis['liquidity_risk']} risk")

        except Exception as e:
            state["errors"].append(f"Forecasting error: {str(e)}")
            state["agent_decisions"]["forecasting"] = {
                "decision": "ERROR",
                "reasoning": f"Processing error: {str(e)}",
                "confidence": 0.0
            }

        return state

    def _supervisor_decision_step(self, state: FinanceWorkflowState) -> FinanceWorkflowState:
        """Supervisor final decision step"""
        print("  🎯 Supervisor Agent: Making final decision...")

        try:
            invoice = Invoice(**state["invoice"])
            vendor = Vendor(**state["vendor_data"])

            # Check escalation criteria
            needs_escalation = supervisor_agent._check_escalation_criteria(invoice, state["agent_decisions"])

            if needs_escalation:
                state["requires_human_approval"] = True
                state["final_decision"] = "REQUIRES_HUMAN"
                state["approval_status"] = "PENDING_HUMAN_APPROVAL"
                decision_reasoning = f"Escalated: Amount ${invoice.amount:,.2f} exceeds threshold or risk criteria met"
            else:
                # Make supervisor decision based on agent inputs
                context = {
                    "workflow_state": state,
                    "agent_decisions": state["agent_decisions"],
                    "risk_assessment": state["risk_assessment"]
                }

                query = f"Make final decision for invoice {invoice.invoice_id} based on agent recommendations."
                supervisor_decision = supervisor_agent.make_decision(context, query)

                state["final_decision"] = supervisor_decision.decision
                state["approval_status"] = supervisor_decision.decision
                decision_reasoning = supervisor_decision.reasoning

                # Store supervisor decision
                state["agent_decisions"]["supervisor"] = asdict(supervisor_decision)

            state["current_step"] = "supervisor_decision"
            state["audit_trail"].append(f"Supervisor: {state['final_decision']} - {decision_reasoning}")

            print(f"     ✓ Supervisor Decision: {state['final_decision']}")
            print(f"     📝 Reasoning: {decision_reasoning}")

        except Exception as e:
            state["errors"].append(f"Supervisor decision error: {str(e)}")
            state["final_decision"] = "ERROR"
            state["approval_status"] = "ERROR"

        return state

    def _human_approval_step(self, state: FinanceWorkflowState) -> FinanceWorkflowState:
        """Human approval simulation step"""
        print("  👤 Human Approval: Simulating human review...")

        # Simulate human approval decision (in real system, this would wait for human input)
        invoice_amount = state["invoice"]["amount"]

        # Simple simulation logic
        if invoice_amount > 50000:
            human_decision = "APPROVED"  # Simulate approval for demo
            reasoning = "High-value transaction approved by CFO after review"
        else:
            human_decision = "APPROVED"  # Simulate approval for demo
            reasoning = "Standard approval after human review"

        state["final_decision"] = human_decision
        state["approval_status"] = human_decision
        state["current_step"] = "human_approval"
        state["audit_trail"].append(f"Human Reviewer: {human_decision} - {reasoning}")

        print(f"     ✓ Human Decision: {human_decision}")
        print(f"     📝 Reasoning: {reasoning}")

        return state

    def _finalize_workflow(self, state: FinanceWorkflowState) -> FinanceWorkflowState:
        """Finalize the workflow"""
        print("  🏁 Finalizing workflow...")

        state["workflow_complete"] = True
        state["current_step"] = "complete"
        state["processing_metadata"]["end_time"] = datetime.now().isoformat()

        # Calculate processing duration
        start_time = datetime.fromisoformat(state["processing_metadata"]["start_time"])
        end_time = datetime.fromisoformat(state["processing_metadata"]["end_time"])
        duration = (end_time - start_time).total_seconds()
        state["processing_metadata"]["duration_seconds"] = duration

        # Update workflow statistics
        self._update_workflow_stats(state)

        state["audit_trail"].append("Workflow completed")

        print(f"     ✅ Final Status: {state['approval_status']}")
        print(f"     ⏱️  Processing Time: {duration:.2f} seconds")

        return state

    # Routing functions
    def _route_after_ap(self, state: FinanceWorkflowState) -> str:
        """Route after AP validation"""
        ap_decision = state["agent_decisions"]["ap"]["decision"]
        return "compliance" if ap_decision in ["APPROVE", "NEEDS_REVIEW"] else "rejected"

    def _route_after_compliance(self, state: FinanceWorkflowState) -> str:
        """Route after compliance check"""
        compliance_decision = state["agent_decisions"]["compliance"]["decision"]

        if compliance_decision == "APPROVE":
            return "payment"
        elif compliance_decision == "HOLD":
            return "hold"
        else:  # NEEDS_REVIEW
            return "human_review"

    def _route_supervisor_decision(self, state: FinanceWorkflowState) -> str:
        """Route after supervisor decision"""
        if state["requires_human_approval"]:
            return "human_review"
        elif state["final_decision"] in ["APPROVE", "APPROVED"]:
            return "approve"
        else:
            return "hold"

    def _update_workflow_stats(self, state: FinanceWorkflowState):
        """Update workflow statistics"""
        self.workflow_stats["total_processed"] += 1

        # Update status counts
        status = state["approval_status"]
        if status in ["APPROVE", "APPROVED"]:
            self.workflow_stats["approved"] += 1
        elif status in ["REJECT", "REJECTED"]:
            self.workflow_stats["rejected"] += 1
        elif status in ["HOLD", "ON_HOLD"]:
            self.workflow_stats["on_hold"] += 1
        elif state["requires_human_approval"]:
            self.workflow_stats["requires_human"] += 1

        # Track processing time
        if "duration_seconds" in state["processing_metadata"]:
            self.workflow_stats["processing_times"].append(state["processing_metadata"]["duration_seconds"])

    def run_workflow(self, invoice: Invoice, po: PurchaseOrder, vendor: Vendor) -> Dict[str, Any]:
        """Execute the complete workflow"""

        # Create initial state
        initial_state: FinanceWorkflowState = {
            "invoice": asdict(invoice),
            "po_data": asdict(po),
            "vendor_data": asdict(vendor),
            "current_step": "",
            "next_agent": "",
            "workflow_complete": False,
            "requires_human_approval": False,
            "agent_decisions": {},
            "agent_messages": [],
            "final_decision": None,
            "approval_status": "PROCESSING",
            "risk_assessment": {},
            "audit_trail": [],
            "processing_metadata": {},
            "errors": [],
            "retry_count": 0
        }

        # Generate unique thread ID for this workflow
        thread_id = f"workflow_{invoice.invoice_id}_{uuid.uuid4().hex[:8]}"

        try:
            # Execute the workflow
            final_state = self.app.invoke(
                initial_state,
                config={"configurable": {"thread_id": thread_id}}
            )

            return final_state

        except Exception as e:
            print(f"❌ Workflow execution error: {str(e)}")
            initial_state["errors"].append(f"Workflow execution error: {str(e)}")
            initial_state["final_decision"] = "ERROR"
            initial_state["approval_status"] = "ERROR"
            return initial_state

    def get_workflow_stats(self) -> Dict[str, Any]:
        """Get workflow statistics"""
        stats = self.workflow_stats.copy()

        if stats["processing_times"]:
            stats["avg_processing_time"] = sum(stats["processing_times"]) / len(stats["processing_times"])
            stats["min_processing_time"] = min(stats["processing_times"])
            stats["max_processing_time"] = max(stats["processing_times"])

        return stats

# Create the workflow graph
workflow_graph = FinanceWorkflowGraph()

print("✅ LangGraph workflow integration completed!")
print("🔄 Workflow nodes configured:")
print("   • start -> ap_validation -> compliance_check -> payment_scheduling")
print("   • -> forecast_update -> supervisor_decision -> [human_approval] -> finalize")
print("🎛️  Conditional routing implemented")
print("📊 Workflow statistics tracking enabled")

**Module 8: Test Scenarios and Execution**

In [ ]:
# Module 8: Test Scenarios and Execution
# Execute different test scenarios and display results

import time
from IPython.display import display, HTML, clear_output
import warnings
warnings.filterwarnings('ignore')

class TestScenarioRunner:
    def __init__(self, workflow_graph, data_generator):
        self.workflow_graph = workflow_graph
        self.data_generator = data_generator
        self.test_results = []

    def run_scenario(self, scenario_name: str, description: str = "") -> Dict[str, Any]:
        """Run a single test scenario"""
        print(f"\n{'='*80}")
        print(f"🧪 TEST SCENARIO: {scenario_name.upper()}")
        print(f"📝 Description: {description}")
        print(f"{'='*80}")

        try:
            # Generate test data for scenario
            invoice, po, vendor = self.data_generator.create_invoice(scenario_name)

            print(f"\n📄 INVOICE DETAILS:")
            print(f"   • Invoice ID: {invoice.invoice_id}")
            print(f"   • Vendor: {vendor.name} (Risk: {vendor.risk_rating})")
            print(f"   • Amount: ${invoice.amount:,.2f}")
            print(f"   • Due Date: {invoice.due_date}")
            print(f"   • PO Number: {invoice.po_number}")

            # Execute workflow
            start_time = time.time()
            result = self.workflow_graph.run_workflow(invoice, po, vendor)
            execution_time = time.time() - start_time

            # Store test result
            test_result = {
                "scenario": scenario_name,
                "description": description,
                "invoice_id": invoice.invoice_id,
                "vendor_name": vendor.name,
                "amount": invoice.amount,
                "final_decision": result["final_decision"],
                "approval_status": result["approval_status"],
                "requires_human": result["requires_human_approval"],
                "execution_time": execution_time,
                "agent_decisions": result["agent_decisions"],
                "audit_trail": result["audit_trail"],
                "errors": result["errors"],
                "result": result
            }

            self.test_results.append(test_result)

            # Display results
            self._display_scenario_results(test_result)

            return test_result

        except Exception as e:
            print(f"❌ Scenario execution failed: {str(e)}")
            return {"error": str(e), "scenario": scenario_name}

    def _display_scenario_results(self, test_result: Dict[str, Any]):
        """Display formatted results for a scenario"""
        result = test_result["result"]

        print(f"\n🎯 WORKFLOW RESULTS:")
        print(f"   • Final Decision: {result['final_decision']}")
        print(f"   • Status: {result['approval_status']}")
        print(f"   • Human Approval Required: {'Yes' if result['requires_human_approval'] else 'No'}")
        print(f"   • Processing Time: {test_result['execution_time']:.2f} seconds")
        print(f"   • Errors: {len(result['errors'])} errors")

        print(f"\n🤖 AGENT DECISIONS:")
        for agent_name, decision_data in result["agent_decisions"].items():
            confidence = decision_data.get("confidence", 0)
            print(f"   • {agent_name.upper()}: {decision_data['decision']} (confidence: {confidence:.2f})")
            if len(decision_data.get("reasoning", "")) > 100:
                reasoning = decision_data["reasoning"][:100] + "..."
            else:
                reasoning = decision_data.get("reasoning", "No reasoning provided")
            print(f"     └─ {reasoning}")

        print(f"\n📋 AUDIT TRAIL:")
        for i, entry in enumerate(result["audit_trail"][-5:], 1):  # Show last 5 entries
            print(f"   {i}. {entry}")

        if result["errors"]:
            print(f"\n⚠️  ERRORS:")
            for error in result["errors"]:
                print(f"   • {error}")

        # Risk Assessment Summary
        if "risk_assessment" in result:
            print(f"\n🛡️  RISK ASSESSMENT:")
            risk_data = result["risk_assessment"]
            for key, value in risk_data.items():
                if isinstance(value, dict):
                    print(f"   • {key}: {json.dumps(value, indent=6)}")
                else:
                    print(f"   • {key}: {value}")

    def run_all_scenarios(self):
        """Run all predefined test scenarios"""
        scenarios = [
            ("clean", "Standard invoice with perfect PO match - should auto-approve"),
            ("po_mismatch", "Invoice amount exceeds PO - should require reconciliation"),
            ("high_value", "High-value transaction - should require human approval"),
            ("bank_change", "Recent bank details change - should trigger verification"),
            ("risk_vendor", "High-risk vendor transaction - should require additional checks")
        ]

        print(f"🚀 EXECUTING ALL TEST SCENARIOS")
        print(f"📊 Running {len(scenarios)} scenarios...")

        for scenario, description in scenarios:
            self.run_scenario(scenario, description)
            time.sleep(1)  # Brief pause between scenarios

        # Display summary
        self._display_test_summary()

    def _display_test_summary(self):
        """Display summary of all test results"""
        print(f"\n{'='*80}")
        print(f"📊 TEST EXECUTION SUMMARY")
        print(f"{'='*80}")

        if not self.test_results:
            print("No test results available.")
            return

        # Summary statistics
        total_tests = len(self.test_results)
        approved = sum(1 for r in self.test_results if r["final_decision"] in ["APPROVE", "APPROVED"])
        rejected = sum(1 for r in self.test_results if r["final_decision"] in ["REJECT", "REJECTED"])
        on_hold = sum(1 for r in self.test_results if r["final_decision"] in ["HOLD", "ON_HOLD"])
        requires_human = sum(1 for r in self.test_results if r["requires_human"])
        errors = sum(1 for r in self.test_results if r["result"]["errors"])

        avg_time = sum(r["execution_time"] for r in self.test_results) / total_tests

        print(f"\n📈 OVERALL STATISTICS:")
        print(f"   • Total Tests: {total_tests}")
        print(f"   • Approved: {approved} ({approved/total_tests*100:.1f}%)")
        print(f"   • Rejected: {rejected} ({rejected/total_tests*100:.1f}%)")
        print(f"   • On Hold: {on_hold} ({on_hold/total_tests*100:.1f}%)")
        print(f"   • Human Review Required: {requires_human} ({requires_human/total_tests*100:.1f}%)")
        print(f"   • Errors: {errors} ({errors/total_tests*100:.1f}%)")
        print(f"   • Average Processing Time: {avg_time:.2f} seconds")

        print(f"\n🎯 SCENARIO BREAKDOWN:")
        for result in self.test_results:
            status_icon = "✅" if result["final_decision"] in ["APPROVE", "APPROVED"] else "⚠️" if result["requires_human"] else "❌"
            print(f"   {status_icon} {result['scenario'].upper()}: {result['final_decision']} - ${result['amount']:,.2f}")

        # Agent Performance Summary
        self._display_agent_performance()

    def _display_agent_performance(self):
        """Display agent performance metrics"""
        print(f"\n🤖 AGENT PERFORMANCE ANALYSIS:")

        agent_stats = {}

        # Collect agent statistics
        for result in self.test_results:
            for agent_name, decision_data in result["agent_decisions"].items():
                if agent_name not in agent_stats:
                    agent_stats[agent_name] = {
                        "total_decisions": 0,
                        "approvals": 0,
                        "rejections": 0,
                        "holds": 0,
                        "confidence_scores": [],
                        "avg_confidence": 0
                    }

                stats = agent_stats[agent_name]
                stats["total_decisions"] += 1
                stats["confidence_scores"].append(decision_data.get("confidence", 0))

                decision = decision_data["decision"]
                if decision in ["APPROVE", "APPROVED"]:
                    stats["approvals"] += 1
                elif decision in ["REJECT", "REJECTED"]:
                    stats["rejections"] += 1
                else:
                    stats["holds"] += 1

        # Calculate averages and display
        for agent_name, stats in agent_stats.items():
            if stats["confidence_scores"]:
                stats["avg_confidence"] = sum(stats["confidence_scores"]) / len(stats["confidence_scores"])

            print(f"\n   🔹 {agent_name.upper()} AGENT:")
            print(f"      • Total Decisions: {stats['total_decisions']}")
            print(f"      • Approvals: {stats['approvals']} ({stats['approvals']/stats['total_decisions']*100:.1f}%)")
            print(f"      • Average Confidence: {stats['avg_confidence']:.2f}")
            print(f"      • Decision Distribution: {stats['approvals']}A / {stats['rejections']}R / {stats['holds']}H")

class ScenarioAnalyzer:
    def __init__(self, test_results):
        self.test_results = test_results

    def analyze_decision_patterns(self):
        """Analyze decision patterns across scenarios"""
        print(f"\n🔍 DECISION PATTERN ANALYSIS:")

        # Pattern 1: Amount vs Decision
        print(f"\n💰 Amount vs Decision Pattern:")
        for result in sorted(self.test_results, key=lambda x: x["amount"]):
            amount = result["amount"]
            decision = result["final_decision"]
            human_required = "👤" if result["requires_human"] else "🤖"
            print(f"   ${amount:>8,.2f} → {decision:<12} {human_required}")

        # Pattern 2: Vendor Risk vs Outcome
        print(f"\n🛡️  Vendor Risk vs Outcome:")
        risk_outcomes = {}
        for result in self.test_results:
            vendor_data = result["result"]["vendor_data"]
            risk_rating = vendor_data["risk_rating"]
            decision = result["final_decision"]

            if risk_rating not in risk_outcomes:
                risk_outcomes[risk_rating] = {"total": 0, "approved": 0, "human_review": 0}

            risk_outcomes[risk_rating]["total"] += 1
            if decision in ["APPROVE", "APPROVED"]:
                risk_outcomes[risk_rating]["approved"] += 1
            if result["requires_human"]:
                risk_outcomes[risk_rating]["human_review"] += 1

        for risk, outcomes in risk_outcomes.items():
            approval_rate = outcomes["approved"] / outcomes["total"] * 100
            human_rate = outcomes["human_review"] / outcomes["total"] * 100
            print(f"   {risk:<8} Risk: {approval_rate:>5.1f}% approved, {human_rate:>5.1f}% human review")

        # Pattern 3: Agent Consensus Analysis
        print(f"\n🤝 Agent Consensus Analysis:")
        consensus_count = 0
        for result in self.test_results:
            decisions = [d["decision"] for d in result["agent_decisions"].values()]
            unique_decisions = set(decisions)
            is_consensus = len(unique_decisions) <= 2  # Allow for minor variations

            if is_consensus:
                consensus_count += 1

            consensus_icon = "✅" if is_consensus else "🔄"
            print(f"   {consensus_icon} {result['scenario']}: {len(unique_decisions)} different decisions")

        print(f"\nConsensus Rate: {consensus_count}/{len(self.test_results)} ({consensus_count/len(self.test_results)*100:.1f}%)")

# Initialize test runner
test_runner = TestScenarioRunner(workflow_graph, data_generator)

print("✅ Test Scenario Runner initialized!")
print("🧪 Available test scenarios:")
print("   • clean: Standard clean invoice")
print("   • po_mismatch: Invoice-PO amount discrepancy")
print("   • high_value: High-value transaction")
print("   • bank_change: Recent bank details change")
print("   • risk_vendor: High-risk vendor")
print("\n🚀 Ready to execute test scenarios!")

**Execute Test Scenarios**

In [ ]:
# Execute all test scenarios
print("🚀 STARTING COMPREHENSIVE TEST EXECUTION")
print("=" * 80)

# Run all scenarios
test_runner.run_all_scenarios()

# Get workflow statistics from the graph
workflow_stats = workflow_graph.get_workflow_stats()
print(f"\n📊 WORKFLOW ENGINE STATISTICS:")
print(f"   • Total Workflows Processed: {workflow_stats['total_processed']}")
print(f"   • Approved: {workflow_stats['approved']}")
print(f"   • Rejected: {workflow_stats['rejected']}")
print(f"   • On Hold: {workflow_stats['on_hold']}")
print(f"   • Requires Human: {workflow_stats['requires_human']}")

if workflow_stats['processing_times']:
    print(f"   • Average Processing Time: {workflow_stats['avg_processing_time']:.2f} seconds")
    print(f"   • Min/Max Processing Time: {workflow_stats['min_processing_time']:.2f}s / {workflow_stats['max_processing_time']:.2f}s")

print(f"\n🔍 RUNNING DECISION PATTERN ANALYSIS...")
analyzer = ScenarioAnalyzer(test_runner.test_results)
analyzer.analyze_decision_patterns()

**Individual Scenario Testing**

In [ ]:
# Test individual scenarios for detailed analysis
print("\n" + "="*80)
print("🎯 INDIVIDUAL SCENARIO DEEP DIVE")
print("="*80)

# Test 1: Clean scenario - should be fastest and auto-approved
print("\n🧪 TESTING: Clean Invoice Scenario")
clean_result = test_runner.run_scenario("clean", "Perfect invoice match - baseline test")

# Test 2: High-value scenario - should trigger human approval
print("\n🧪 TESTING: High-Value Invoice Scenario")
high_value_result = test_runner.run_scenario("high_value", "Transaction above approval threshold")

# Test 3: PO Mismatch - should require reconciliation
print("\n🧪 TESTING: PO Mismatch Scenario")
mismatch_result = test_runner.run_scenario("po_mismatch", "Invoice amount exceeds PO amount")

print(f"\n🎯 SCENARIO COMPARISON:")
print(f"   • Clean Invoice: {clean_result['final_decision']} in {clean_result['execution_time']:.2f}s")
print(f"   • High Value: {high_value_result['final_decision']} in {high_value_result['execution_time']:.2f}s")
print(f"   • PO Mismatch: {mismatch_result['final_decision']} in {mismatch_result['execution_time']:.2f}s")

**Custom Scenario Testing**

In [ ]:
# Custom scenario testing function
def run_custom_scenario(vendor_id: str, amount: float, scenario_type: str = "clean"):
    """Run a custom scenario with specific parameters"""
    print(f"\n🎯 CUSTOM SCENARIO TEST")
    print(f"   • Vendor ID: {vendor_id}")
    print(f"   • Amount: ${amount:,.2f}")
    print(f"   • Base Scenario: {scenario_type}")

    # Generate base scenario
    invoice, po, vendor = data_generator.create_invoice(scenario_type)

    # Customize parameters
    if vendor_id in data_generator.vendors:
        vendor = data_generator.vendors[vendor_id]
        invoice.vendor_id = vendor.vendor_id
        invoice.vendor_name = vendor.name
        invoice.bank_details = vendor.bank_details
        po.vendor_id = vendor.vendor_id

    # Customize amount
    invoice.amount = amount
    po.total_amount = amount

    # Update line items proportionally
    if invoice.line_items:
        scale_factor = amount / sum(item["total"] for item in invoice.line_items)
        for item in invoice.line_items:
            item["total"] = item["total"] * scale_factor
            item["unit_price"] = item["unit_price"] * scale_factor

        # Update PO line items to match
        if po.line_items:
            for i, item in enumerate(po.line_items):
                if i < len(invoice.line_items):
                    item["total"] = invoice.line_items[i]["total"]
                    item["unit_price"] = invoice.line_items[i]["unit_price"]

    # Run workflow
    result = workflow_graph.run_workflow(invoice, po, vendor)

    print(f"   • Result: {result['final_decision']}")
    print(f"   • Human Required: {result['requires_human_approval']}")
    print(f"   • Processing Time: {(time.time() - time.time()):.2f}s")  # Placeholder

    return result

# Test custom scenarios
print(f"\n🧪 CUSTOM SCENARIO TESTS:")

# High-value with low-risk vendor
custom1 = run_custom_scenario("VEN001", 30000, "clean")

# Medium amount with high-risk vendor
custom2 = run_custom_scenario("VEN003", 8000, "risk_vendor")

# Very high amount with medium-risk vendor
custom3 = run_custom_scenario("VEN002", 100000, "high_value")

print(f"\n📊 CUSTOM TEST SUMMARY:")
results = [custom1, custom2, custom3]
for i, result in enumerate(results, 1):
    status = "✅" if result["final_decision"] in ["APPROVE", "APPROVED"] else "⚠️"
    print(f"   Custom Test {i}: {status} {result['final_decision']}")

**Module 9: Analytics Dashboard and Visualization**

In [ ]:
# Module 9: Analytics Dashboard and Visualization
# Create comprehensive visual analytics for the agentic finance system

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import json

class FinanceAnalyticsDashboard:
    def __init__(self, test_results, workflow_stats):
        self.test_results = test_results
        self.workflow_stats = workflow_stats
        self.df = self._create_analytics_dataframe()

    def _create_analytics_dataframe(self):
        """Create a comprehensive DataFrame from test results"""
        data = []

        for result in self.test_results:
            # Extract key metrics
            row = {
                'scenario': result['scenario'],
                'invoice_id': result['invoice_id'],
                'vendor_name': result['vendor_name'],
                'amount': result['amount'],
                'final_decision': result['final_decision'],
                'approval_status': result['approval_status'],
                'requires_human': result['requires_human'],
                'execution_time': result['execution_time'],
                'error_count': len(result['errors'])
            }

            # Extract vendor data
            vendor_data = result['result']['vendor_data']
            row.update({
                'vendor_risk': vendor_data['risk_rating'],
                'payment_terms': vendor_data['payment_terms'],
                'has_discount': vendor_data.get('discount_terms') is not None
            })

            # Extract agent decisions and confidence scores
            for agent_name, decision_data in result['agent_decisions'].items():
                row[f'{agent_name}_decision'] = decision_data['decision']
                row[f'{agent_name}_confidence'] = decision_data.get('confidence', 0)

            # Extract risk assessment data
            risk_data = result['result'].get('risk_assessment', {})
            row.update({
                'liquidity_risk': risk_data.get('liquidity_analysis', {}).get('liquidity_risk', 'UNKNOWN'),
                'sanctions_clear': risk_data.get('sanctions_clear', True),
                'bank_change_risk': risk_data.get('bank_validation', {}).get('risk_score', 1)
            })

            data.append(row)

        return pd.DataFrame(data)

    def create_executive_dashboard(self):
        """Create executive-level dashboard with key KPIs"""
        print("📊 EXECUTIVE DASHBOARD")
        print("=" * 60)

        # Create subplots
        fig = make_subplots(
            rows=2, cols=3,
            subplot_titles=(
                'Decision Distribution', 'Processing Times', 'Risk vs Decision',
                'Agent Performance', 'Amount Distribution', 'Human Intervention Rate'
            ),
            specs=[
                [{"type": "pie"}, {"type": "histogram"}, {"type": "scatter"}],
                [{"type": "bar"}, {"type": "box"}, {"type": "indicator"}]
            ]
        )

        # 1. Decision Distribution (Pie Chart)
        decision_counts = self.df['final_decision'].value_counts()
        fig.add_trace(
            go.Pie(
                labels=decision_counts.index,
                values=decision_counts.values,
                name="Decisions",
                marker=dict(colors=['#2E8B57', '#DC143C', '#FF8C00', '#4169E1'])
            ),
            row=1, col=1
        )

        # 2. Processing Times (Histogram)
        fig.add_trace(
            go.Histogram(
                x=self.df['execution_time'],
                name="Processing Time",
                nbinsx=10,
                marker_color='#4CAF50'
            ),
            row=1, col=2
        )

        # 3. Risk vs Decision (Scatter)
        risk_mapping = {'LOW': 1, 'MEDIUM': 2, 'HIGH': 3}
        decision_mapping = {'APPROVED': 1, 'APPROVE': 1, 'HOLD': 2, 'REJECTED': 3, 'REQUIRES_HUMAN': 2.5}

        fig.add_trace(
            go.Scatter(
                x=[risk_mapping.get(risk, 1) for risk in self.df['vendor_risk']],
                y=[decision_mapping.get(dec, 2) for dec in self.df['final_decision']],
                mode='markers',
                marker=dict(
                    size=self.df['amount']/1000,  # Size by amount
                    color=self.df['execution_time'],
                    colorscale='Viridis',
                    showscale=True
                ),
                text=self.df['scenario'],
                name="Risk Analysis"
            ),
            row=1, col=3
        )

        # 4. Agent Performance (Bar Chart)
        agent_cols = [col for col in self.df.columns if col.endswith('_confidence')]
        agent_names = [col.replace('_confidence', '').upper() for col in agent_cols]
        avg_confidence = [self.df[col].mean() for col in agent_cols]

        fig.add_trace(
            go.Bar(
                x=agent_names,
                y=avg_confidence,
                name="Avg Confidence",
                marker_color='#FF6B6B'
            ),
            row=2, col=1
        )

        # 5. Amount Distribution (Box Plot)
        fig.add_trace(
            go.Box(
                y=self.df['amount'],
                name="Amount Distribution",
                marker_color='#4ECDC4'
            ),
            row=2, col=2
        )

        # 6. Human Intervention Rate (Gauge)
        human_rate = (self.df['requires_human'].sum() / len(self.df)) * 100
        fig.add_trace(
            go.Indicator(
                mode="gauge+number+delta",
                value=human_rate,
                domain={'x': [0, 1], 'y': [0, 1]},
                title={'text': "Human Intervention %"},
                gauge={
                    'axis': {'range': [None, 100]},
                    'bar': {'color': "#FF6B6B"},
                    'steps': [
                        {'range': [0, 25], 'color': "#90EE90"},
                        {'range': [25, 50], 'color': "#FFD700"},
                        {'range': [50, 100], 'color': "#FF6B6B"}
                    ],
                    'threshold': {
                        'line': {'color': "red", 'width': 4},
                        'thickness': 0.75,
                        'value': 30
                    }
                }
            ),
            row=2, col=3
        )

        # Update layout
        fig.update_layout(
            height=800,
            title_text="Finance AI Agent System - Executive Dashboard",
            title_x=0.5,
            showlegend=True
        )

        fig.show()

        # Display key metrics
        self._display_executive_metrics()

    def _display_executive_metrics(self):
        """Display key executive metrics"""
        total_processed = len(self.df)
        total_amount = self.df['amount'].sum()
        avg_processing_time = self.df['execution_time'].mean()

        approved_rate = (self.df['final_decision'].isin(['APPROVE', 'APPROVED']).sum() / total_processed) * 100
        human_intervention_rate = (self.df['requires_human'].sum() / total_processed) * 100
        error_rate = (self.df['error_count'] > 0).sum() / total_processed * 100

        print(f"\n📈 KEY PERFORMANCE INDICATORS:")
        print(f"   • Total Transactions Processed: {total_processed}")
        print(f"   • Total Transaction Value: ${total_amount:,.2f}")
        print(f"   • Approval Rate: {approved_rate:.1f}%")
        print(f"   • Human Intervention Rate: {human_intervention_rate:.1f}%")
        print(f"   • Average Processing Time: {avg_processing_time:.2f} seconds")
        print(f"   • Error Rate: {error_rate:.1f}%")
        print(f"   • Straight-Through Processing: {100 - human_intervention_rate:.1f}%")

        # Risk-based metrics
        high_risk_processed = (self.df['vendor_risk'] == 'HIGH').sum()
        high_risk_approved = self.df[(self.df['vendor_risk'] == 'HIGH') &
                                   (self.df['final_decision'].isin(['APPROVE', 'APPROVED']))].shape[0]

        if high_risk_processed > 0:
            high_risk_approval_rate = (high_risk_approved / high_risk_processed) * 100
            print(f"   • High-Risk Vendor Approval Rate: {high_risk_approval_rate:.1f}%")

    def create_agent_performance_analysis(self):
        """Detailed agent performance analysis"""
        print(f"\n🤖 AGENT PERFORMANCE DEEP DIVE")
        print("=" * 60)

        # Agent confidence distribution
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Agent Confidence Distributions', 'Decision Agreement Matrix',
                'Agent Response Times', 'Confidence vs Accuracy'
            )
        )

        # 1. Confidence distributions for each agent
        agent_cols = [col for col in self.df.columns if col.endswith('_confidence')]

        for i, col in enumerate(agent_cols):
            agent_name = col.replace('_confidence', '').upper()
            fig.add_trace(
                go.Violin(
                    y=self.df[col],
                    name=f"{agent_name}",
                    box_visible=True,
                    meanline_visible=True
                ),
                row=1, col=1
            )

        # 2. Decision agreement heatmap
        decision_cols = [col for col in self.df.columns if col.endswith('_decision')]
        if len(decision_cols) > 1:
            # Create agreement matrix
            decisions_df = self.df[decision_cols]

            # Convert decisions to numeric for correlation
            decision_mapping = {'APPROVE': 1, 'APPROVED': 1, 'HOLD': 0, 'REJECT': -1, 'NEEDS_REVIEW': 0.5}
            decisions_numeric = decisions_df.replace(decision_mapping)
            correlation_matrix = decisions_numeric.corr()

            fig.add_trace(
                go.Heatmap(
                    z=correlation_matrix.values,
                    x=[col.replace('_decision', '').upper() for col in correlation_matrix.columns],
                    y=[col.replace('_decision', '').upper() for col in correlation_matrix.index],
                    colorscale='RdBu',
                    zmid=0
                ),
                row=1, col=2
            )

        # 3. Processing time impact (mock data since we don't have individual agent times)
        agent_names = [col.replace('_confidence', '').upper() for col in agent_cols]
        mock_times = np.random.normal(0.5, 0.1, len(agent_names))  # Mock processing times

        fig.add_trace(
            go.Bar(
                x=agent_names,
                y=mock_times,
                name="Avg Processing Time",
                marker_color='#9B59B6'
            ),
            row=2, col=1
        )

        # 4. Confidence vs Accuracy scatter
        for col in agent_cols:
            agent_name = col.replace('_confidence', '').upper()
            decision_col = col.replace('_confidence', '_decision')

            if decision_col in self.df.columns:
                # Mock accuracy calculation (would be based on actual outcomes)
                accuracy = np.random.uniform(0.7, 0.95, len(self.df))

                fig.add_trace(
                    go.Scatter(
                        x=self.df[col],
                        y=accuracy,
                        mode='markers',
                        name=f"{agent_name}",
                        marker=dict(size=8)
                    ),
                    row=2, col=2
                )

        fig.update_layout(
            height=800,
            title_text="Agent Performance Analysis",
            title_x=0.5
        )

        fig.show()

        # Agent-specific statistics
        self._display_agent_statistics()

    def _display_agent_statistics(self):
        """Display detailed agent statistics"""
        agent_cols = [col for col in self.df.columns if col.endswith('_confidence')]

        print(f"\n📊 INDIVIDUAL AGENT STATISTICS:")

        for col in agent_cols:
            agent_name = col.replace('_confidence', '').upper()
            decision_col = col.replace('_confidence', '_decision')

            confidence_stats = self.df[col].describe()

            print(f"\n🔹 {agent_name} AGENT:")
            print(f"   • Average Confidence: {confidence_stats['mean']:.3f}")
            print(f"   • Confidence Range: {confidence_stats['min']:.3f} - {confidence_stats['max']:.3f}")
            print(f"   • Std Deviation: {confidence_stats['std']:.3f}")

            if decision_col in self.df.columns:
                decision_counts = self.df[decision_col].value_counts()
                total_decisions = len(self.df)
                print(f"   • Decision Distribution:")
                for decision, count in decision_counts.items():
                    percentage = (count / total_decisions) * 100
                    print(f"     - {decision}: {count} ({percentage:.1f}%)")

                # High confidence decisions
                high_conf_threshold = 0.8
                high_conf_decisions = (self.df[col] >= high_conf_threshold).sum()
                print(f"   • High Confidence Decisions (≥{high_conf_threshold}): {high_conf_decisions}/{total_decisions} ({high_conf_decisions/total_decisions*100:.1f}%)")

    def create_business_impact_analysis(self):
        """Analyze business impact and financial metrics"""
        print(f"\n💼 BUSINESS IMPACT ANALYSIS")
        print("=" * 60)

        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Transaction Value by Decision', 'Risk-Adjusted Processing',
                'Discount Capture Opportunity', 'Processing Efficiency Trends'
            ),
            specs=[
                [{"secondary_y": False}, {"type": "scatter"}],
                [{"type": "bar"}, {"type": "scatter"}]
            ]
        )

        # 1. Transaction value by decision type
        decision_amounts = self.df.groupby('final_decision')['amount'].agg(['sum', 'count', 'mean'])

        fig.add_trace(
            go.Bar(
                x=decision_amounts.index,
                y=decision_amounts['sum'],
                name="Total Value",
                marker_color='#2E8B57',
                text=[f"${val:,.0f}" for val in decision_amounts['sum']],
                textposition='outside'
            ),
            row=1, col=1
        )

        # 2. Risk-adjusted processing (Risk vs Amount vs Decision)
        risk_colors = {'LOW': '#2E8B57', 'MEDIUM': '#FF8C00', 'HIGH': '#DC143C'}

        for risk in self.df['vendor_risk'].unique():
            risk_data = self.df[self.df['vendor_risk'] == risk]
            fig.add_trace(
                go.Scatter(
                    x=risk_data['amount'],
                    y=risk_data['execution_time'],
                    mode='markers',
                    name=f"{risk} Risk",
                    marker=dict(
                        color=risk_colors.get(risk, '#666666'),
                        size=10,
                        opacity=0.7
                    ),
                    text=risk_data['scenario']
                ),
                row=1, col=2
            )

        # 3. Discount capture opportunities
        discount_data = self.df[self.df['has_discount'] == True]
        if len(discount_data) > 0:
            # Mock discount calculations
            potential_savings = discount_data['amount'] * 0.02  # Assume 2% average discount
            fig.add_trace(
                go.Bar(
                    x=discount_data['scenario'],
                    y=potential_savings,
                    name="Potential Savings",
                    marker_color='#4CAF50'
                ),
                row=2, col=1
            )

        # 4. Processing efficiency trends
        self.df['efficiency_score'] = (1 - self.df['execution_time'] / self.df['execution_time'].max()) * 100

        fig.add_trace(
            go.Scatter(
                x=self.df['amount'],
                y=self.df['efficiency_score'],
                mode='markers+lines',
                name="Efficiency Score",
                marker=dict(
                    color=self.df['execution_time'],
                    colorscale='RdYlGn',
                    showscale=True
                ),
                text=self.df['scenario']
            ),
            row=2, col=2
        )

        fig.update_layout(
            height=800,
            title_text="Business Impact Analysis",
            title_x=0.5
        )

        fig.show()

        # Calculate business metrics
        self._display_business_metrics()

    def _display_business_metrics(self):
        """Display key business impact metrics"""
        total_value = self.df['amount'].sum()
        approved_value = self.df[self.df['final_decision'].isin(['APPROVE', 'APPROVED'])]['amount'].sum()

        # Calculate potential savings from discounts
        discount_eligible = self.df[self.df['has_discount'] == True]
        potential_discount_savings = discount_eligible['amount'].sum() * 0.02  # Assume 2% average

        # Processing cost savings (assume $50 per manual process)
        manual_processes_avoided = len(self.df) - self.df['requires_human'].sum()
        processing_cost_savings = manual_processes_avoided * 50

        print(f"\n💰 FINANCIAL IMPACT METRICS:")
        print(f"   • Total Transaction Value Processed: ${total_value:,.2f}")
        print(f"   • Approved Transaction Value: ${approved_value:,.2f}")
        print(f"   • Potential Discount Savings: ${potential_discount_savings:,.2f}")
        print(f"   • Processing Cost Savings: ${processing_cost_savings:,.2f}")
        print(f"   • Manual Processes Avoided: {manual_processes_avoided}/{len(self.df)}")

        # Risk metrics
        high_risk_amount = self.df[self.df['vendor_risk'] == 'HIGH']['amount'].sum()
        high_risk_approved = self.df[(self.df['vendor_risk'] == 'HIGH') &
                                   (self.df['final_decision'].isin(['APPROVE', 'APPROVED']))]['amount'].sum()

        print(f"\n🛡️  RISK MANAGEMENT METRICS:")
        print(f"   • High-Risk Transaction Volume: ${high_risk_amount:,.2f}")
        print(f"   • High-Risk Approved Amount: ${high_risk_approved:,.2f}")
        if high_risk_amount > 0:
            print(f"   • High-Risk Approval Rate: {(high_risk_approved/high_risk_amount)*100:.1f}%")

    def create_workflow_optimization_report(self):
        """Create workflow optimization recommendations"""
        print(f"\n⚡ WORKFLOW OPTIMIZATION REPORT")
        print("=" * 60)

        # Identify bottlenecks and optimization opportunities
        avg_processing_time = self.df['execution_time'].mean()
        slow_scenarios = self.df[self.df['execution_time'] > avg_processing_time * 1.5]

        print(f"🔍 PERFORMANCE ANALYSIS:")
        print(f"   • Average Processing Time: {avg_processing_time:.2f} seconds")
        print(f"   • Slow Processing Scenarios: {len(slow_scenarios)}/{len(self.df)}")

        if len(slow_scenarios) > 0:
            print(f"   • Slowest Scenarios:")
            for _, row in slow_scenarios.iterrows():
                print(f"     - {row['scenario']}: {row['execution_time']:.2f}s (${row['amount']:,.2f})")

        # Agent performance optimization
        agent_cols = [col for col in self.df.columns if col.endswith('_confidence')]
        low_confidence_threshold = 0.7

        print(f"\n🤖 AGENT OPTIMIZATION OPPORTUNITIES:")
        for col in agent_cols:
            agent_name = col.replace('_confidence', '').upper()
            low_confidence_count = (self.df[col] < low_confidence_threshold).sum()

            if low_confidence_count > 0:
                percentage = (low_confidence_count / len(self.df)) * 100
                print(f"   • {agent_name}: {low_confidence_count} low-confidence decisions ({percentage:.1f}%)")

                # Identify scenarios where this agent struggles
                low_conf_scenarios = self.df[self.df[col] < low_confidence_threshold]['scenario'].value_counts()
                if len(low_conf_scenarios) > 0:
                    print(f"     - Challenging scenarios: {', '.join(low_conf_scenarios.index[:3])}")

        # Business rule optimization
        human_intervention_scenarios = self.df[self.df['requires_human'] == True]
        if len(human_intervention_scenarios) > 0:
            print(f"\n👤 HUMAN INTERVENTION ANALYSIS:")
            print(f"   • Total requiring human review: {len(human_intervention_scenarios)}")

            intervention_by_scenario = human_intervention_scenarios['scenario'].value_counts()
            print(f"   • By scenario type:")
            for scenario, count in intervention_by_scenario.items():
                print(f"     - {scenario}: {count} cases")

            # Amount-based analysis
            intervention_amounts = human_intervention_scenarios['amount']
            print(f"   • Amount distribution requiring review:")
            print(f"     - Average: ${intervention_amounts.mean():,.2f}")
            print(f"     - Range: ${intervention_amounts.min():,.2f} - ${intervention_amounts.max():,.2f}")

        # Generate recommendations
        self._generate_optimization_recommendations()

    def _generate_optimization_recommendations(self):
        """Generate specific optimization recommendations"""
        print(f"\n💡 OPTIMIZATION RECOMMENDATIONS:")

        recommendations = []

        # Processing time recommendations
        avg_time = self.df['execution_time'].mean()
        if avg_time > 2.0:
            recommendations.append("Consider optimizing agent response times - average processing exceeds 2 seconds")

        # Human intervention rate
        human_rate = (self.df['requires_human'].sum() / len(self.df)) * 100
        if human_rate > 30:
            recommendations.append(f"High human intervention rate ({human_rate:.1f}%) - review approval thresholds")
        elif human_rate < 10:
            recommendations.append("Very low human intervention - consider raising automation thresholds carefully")

        # Agent confidence
        agent_cols = [col for col in self.df.columns if col.endswith('_confidence')]
        for col in agent_cols:
            avg_conf = self.df[col].mean()
            agent_name = col.replace('_confidence', '').upper()
            if avg_conf < 0.75:
                recommendations.append(f"{agent_name} agent shows low average confidence ({avg_conf:.2f}) - retrain or adjust prompts")

        # Risk-based recommendations
        high_risk_auto = self.df[(self.df['vendor_risk'] == 'HIGH') &
                                (self.df['requires_human'] == False)]
        if len(high_risk_auto) > 0:
            recommendations.append("Some high-risk vendors processed automatically - review risk thresholds")

        # Display recommendations
        if recommendations:
            for i, rec in enumerate(recommendations, 1):
                print(f"   {i}. {rec}")
        else:
            print("   ✅ No critical optimization opportunities identified - system performing well!")

        # Success metrics
        print(f"\n🎯 SYSTEM STRENGTHS:")
        strengths = []

        if self.df['error_count'].sum() == 0:
            strengths.append("Zero processing errors across all scenarios")

        if (self.df['execution_time'] < 3.0).all():
            strengths.append("All transactions processed within 3 seconds")

        high_conf_rate = sum((self.df[col] >= 0.8).sum() for col in agent_cols) / (len(self.df) * len(agent_cols))
        if high_conf_rate > 0.7:
            strengths.append(f"High agent confidence rate ({high_conf_rate:.1%})")

        for strength in strengths:
            print(f"   ✅ {strength}")

    def export_analytics_report(self, filename="finance_ai_analytics_report.html"):
        """Export comprehensive analytics report"""
        html_content = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>Finance AI System Analytics Report</title>
            <style>
                body {{ font-family: Arial, sans-serif; margin: 20px; }}
                .header {{ background-color: #2E8B57; color: white; padding: 20px; text-align: center; }}
                .section {{ margin: 20px 0; padding: 15px; border-left: 4px solid #2E8B57; }}
                .metric {{ display: inline-block; margin: 10px; padding: 10px; background-color: #f0f0f0; border-radius: 5px; }}
                .success {{ color: #2E8B57; }}
                .warning {{ color: #FF8C00; }}
                .error {{ color: #DC143C; }}
            </style>
        </head>
        <body>
            <div class="header">
                <h1>Finance AI Agent System Analytics Report</h1>
                <p>Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
            </div>

            <div class="section">
                <h2>Executive Summary</h2>
                <div class="metric">
                    <strong>Total Processed:</strong> {len(self.df)} transactions
                </div>
                <div class="metric">
                    <strong>Total Value:</strong> ${self.df['amount'].sum():,.2f}
                </div>
                <div class="metric">
                    <strong>Approval Rate:</strong> {(self.df['final_decision'].isin(['APPROVE', 'APPROVED']).sum() / len(self.df)) * 100:.1f}%
                </div>
                <div class="metric">
                    <strong>Avg Processing Time:</strong> {self.df['execution_time'].mean():.2f}s
                </div>
            </div>

            <div class="section">
                <h2>Detailed Analytics</h2>
                {self.df.to_html(classes='table table-striped')}
            </div>
        </body>
        </html>
        """

        with open(filename, 'w') as f:
            f.write(html_content)

        print(f"📄 Analytics report exported to: {filename}")

# Create analytics dashboard
if test_runner.test_results:
    dashboard = FinanceAnalyticsDashboard(test_runner.test_results, workflow_graph.get_workflow_stats())

    print("🎨 CREATING COMPREHENSIVE ANALYTICS DASHBOARD")
    print("=" * 80)

    # Create all dashboard components
    dashboard.create_executive_dashboard()
    dashboard.create_agent_performance_analysis()
    dashboard.create_business_impact_analysis()
    dashboard.create_workflow_optimization_report()

    # Export report
    dashboard.export_analytics_report()

else:
    print("⚠️  No test results available. Please run test scenarios first.")

**Real-time Monitoring Dashboard**

In [ ]:
# Real-time monitoring simulation
class RealTimeMonitor:
    def __init__(self, dashboard):
        self.dashboard = dashboard
        self.alerts = []

    def check_system_health(self):
        """Monitor system health and generate alerts"""
        print(f"\n🚨 REAL-TIME SYSTEM MONITORING")
        print("=" * 50)

        df = self.dashboard.df

        # Performance alerts
        avg_time = df['execution_time'].mean()
        if avg_time > 3.0:
            self.alerts.append({
                'type': 'PERFORMANCE',
                'level': 'WARNING',
                'message': f'Average processing time high: {avg_time:.2f}s',
                'timestamp': datetime.now()
            })

        # Error rate alerts
        error_rate = (df['error_count'] > 0).sum() / len(df)
        if error_rate > 0.05:  # 5% threshold
            self.alerts.append({
                'type': 'ERROR',
                'level': 'CRITICAL',
                'message': f'Error rate exceeded threshold: {error_rate:.1%}',
                'timestamp': datetime.now()
            })

        # Human intervention alerts
        human_rate = df['requires_human'].mean()
        if human_rate > 0.5:  # 50% threshold
            self.alerts.append({
                'type': 'ESCALATION',
                'level': 'WARNING',
                'message': f'High human intervention rate: {human_rate:.1%}',
                'timestamp': datetime.now()
            })

        # Display alerts
        if self.alerts:
            print("🔔 ACTIVE ALERTS:")
            for alert in self.alerts[-5:]:  # Show last 5 alerts
                level_icon = "🚨" if alert['level'] == 'CRITICAL' else "⚠️"
                print(f"   {level_icon} {alert['type']}: {alert['message']}")
                print(f"      └─ {alert['timestamp'].strftime('%H:%M:%S')}")
        else:
            print("✅ No active alerts - system operating normally")

        # System metrics
        print(f"\n📊 CURRENT SYSTEM METRICS:")
        print(f"   • Uptime: 100% (simulated)")
        print(f"   • Throughput: {len(df)} transactions processed")
        print(f"   • Success Rate: {((df['final_decision'] != 'ERROR').sum() / len(df)) * 100:.1f}%")
        print(f"   • Average Response Time: {avg_time:.2f} seconds")

        return len(self.alerts)

    def simulate_live_dashboard(self, duration_seconds=30):
        """Simulate a live monitoring dashboard"""
        print(f"\n📺 LIVE DASHBOARD SIMULATION ({duration_seconds}s)")
        print("=" * 50)

        import time
        import random
        from IPython.display import clear_output

        for i in range(duration_seconds):
            clear_output(wait=True)

            # Simulate real-time metrics
            current_time = datetime.now().strftime("%H:%M:%S")
            transactions_per_second = random.uniform(0.5, 2.0)
            cpu_usage = random.uniform(15, 45)
            memory_usage = random.uniform(25, 65)

            print(f"🔴 LIVE - Finance AI System Dashboard - {current_time}")
            print("=" * 60)
            print(f"📊 Real-time Metrics:")
            print(f"   • Transactions/sec: {transactions_per_second:.1f}")
            print(f"   • CPU Usage: {cpu_usage:.1f}%")
            print(f"   • Memory Usage: {memory_usage:.1f}%")
            print(f"   • Active Workflows: {random.randint(0, 3)}")

            # Simulate processing queue
            queue_size = random.randint(0, 5)
            print(f"\n📋 Processing Queue:")
            print(f"   • Pending: {queue_size} invoices")
            print(f"   • Processing: {random.randint(0, 2)} invoices")
            print(f"   • Awaiting Human Approval: {random.randint(0, 1)} invoices")

            # Simulate recent activity
            activities = [
                "INV-1234 approved automatically",
                "High-value transaction escalated",
                "Vendor VEN001 payment scheduled",
                "Risk check completed for VEN003",
                "Discount captured: $150.00"
            ]

            print(f"\n📝 Recent Activity:")
            for j, activity in enumerate(random.sample(activities, min(3, len(activities))), 1):
                timestamp = (datetime.now() - timedelta(seconds=random.randint(1, 300))).strftime("%H:%M")
                print(f"   {j}. [{timestamp}] {activity}")

            # Alert simulation
            if random.random() < 0.1:  # 10% chance of alert
                alert_types = ["Performance spike detected", "New high-risk vendor", "Discount opportunity available"]
                alert = random.choice(alert_types)
                print(f"\n🚨 ALERT: {alert}")

            time.sleep(1)

        clear_output(wait=True)
        print("📺 Live dashboard simulation completed")

# Create monitoring system
if 'dashboard' in locals():
    monitor = RealTimeMonitor(dashboard)

    # Run system health check
    alert_count = monitor.check_system_health()

    print(f"\n🏥 SYSTEM HEALTH STATUS:")
    if alert_count == 0:
        print("   ✅ HEALTHY - All systems operational")
    elif alert_count <= 2:
        print("   ⚠️  WARNING - Minor issues detected")
    else:
        print("   🚨 CRITICAL - Multiple alerts active")

    # Optional: Run live dashboard simulation (uncomment to enable)
    # print("\n🎬 Starting live dashboard simulation...")
    # monitor.simulate_live_dashboard(10)  # 10 second demo

else:
    print("⚠️  Dashboard not available. Please run previous modules first.")

**Final System Status and Summary**

In [ ]:
# Quick start guide for running the complete system
print("\n📖 QUICK START GUIDE")
print("=" * 50)

print("To run this complete system in Google Colab:")
print("\n1. 📋 Copy and paste each module in order:")
print("   • Module 1: Dependencies and setup (set your OpenAI API key!)")
print("   • Module 2: Data structures and enums")
print("   • Module 3: Mock data generation")
print("   • Module 4: Base agent class")
print("   • Module 5: Specialized agents")
print("   • Module 6: Supervisor agent")
print("   • Module 7: LangGraph workflow")
print("   • Module 8: Test execution")
print("   • Module 9: Analytics dashboard")

print("\n2. 🔧 Customize as needed:")
print("   • Modify approval thresholds in supervisor_agent.escalation_thresholds")
print("   • Adjust agent prompts and decision logic")
print("   • Add new test scenarios in data_generator")
print("   • Customize analytics and visualizations")

print("\n3. 🧪 Run test scenarios:")
print("   test_runner.run_all_scenarios()  # Run all predefined tests")
print("   test_runner.run_scenario('clean')  # Run specific scenario")

print("\n4. 📊 View analytics:")
print("   dashboard.create_executive_dashboard()  # Executive KPIs")
print("   dashboard.create_agent_performance_analysis()  # Agent metrics")
print("   dashboard.create_business_impact_analysis()  # Business value")

print("\n5. 🚀 Extend the system:")
print("   • Add new agents by inheriting from BaseAgent")
print("   • Create custom scenarios with run_custom_scenario()")
print("   • Implement additional analytics and monitoring")

print(f"\n✅ The system is now ready for demonstration and further development!")